In [1]:
import numpy as np 
import pandas as pd
from fastai.data_block import get_files
from fastprogress import master_bar, progress_bar
import tensorflow as tf
import torch 
import os
import pickle

In [ ]:
IN_PATH  = 'yt8m/2/frame/train'
OUT_PATH = 'yt8m-processed'

files = [str(f) for f in get_files(IN_PATH, '.tfrecord')]
files

In [ ]:
raw_dataset = tf.data.TFRecordDataset(files)

In [ ]:
d = dict()
A_SZ, V_SZ = 128,1024
SZ= int(1e7)
a = torch.empty((SZ,A_SZ), dtype=torch.uint8) # audio
v = torch.empty((SZ,V_SZ), dtype=torch.uint8) # video

In [ ]:
i_f = i_av = 0
for i,raw_record in enumerate(progress_bar(raw_dataset.take(-1), total=3888919)):
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    labels = list(example.context.feature['labels'].int64_list.value)
    vid    = str(example.context.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
    audio  = torch.ByteTensor(
        [np.frombuffer(f.bytes_list.value[0], dtype=np.uint8) for f in example.feature_lists.feature_list['audio'].feature])
    video  = torch.ByteTensor(
        [np.frombuffer(f.bytes_list.value[0], dtype=np.uint8) for f in example.feature_lists.feature_list['rgb'].feature])
    frames = len(audio)
    assert frames == len(video)
    d[vid] = (i_av, frames, labels)
    a[i_av:i_av+frames] = audio
    v[i_av:i_av+frames] = video
    del audio, video, labels
    i_av += frames
    if (i_av + 300)>= SZ:
        print(f'Saving {i_f} at {i}')
        torch.save(a[:i_av], f'{OUT_PATH}/a{i_f}.pth')
        torch.save(v[:i_av], f'{OUT_PATH}/v{i_f}.pth')
        with open(f'{OUT_PATH}/d{i_f}.pkl', 'wb') as handle:
            pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
        d = dict()
        i_av = 0
        i_f += 1


In [ ]:
i_f = 89

In [ ]:
def mmap_bytetensor(f, shape, shared=False,size=None):
    return torch.ByteTensor(
        torch.ByteStorage.from_file(f, shared=shared, size=size if size is not None else os.path.getsize(f))).view(*shape)

In [ ]:
ifnone(v.numel(), 10)

In [ ]:
for i in progress_bar(range(89)): # 89
    a = torch.load(f'{OUT_PATH}/a{i}.pth')
    aw = mmap_bytetensor(f'{OUT_PATH}/a{i}.bin',(-1,128),  shared=True, size=a.numel())
    aw[:] = a
    v = torch.load(f'{OUT_PATH}/v{i}.pth')
    vw = mmap_bytetensor(f'{OUT_PATH}/v{i}.bin',(-1,1024), shared=True, size=v.numel())
    vw[:] = v
    os.remove(f'{OUT_PATH}/a{i}.pth')
    os.remove(f'{OUT_PATH}/v{i}.pth')

In [ ]:
vvw=read_bytetensor('yt8m-processed/v0.bin', (-1,1024))

In [ ]:
n = 89
[read_bytetensor(f'yt8m-processed/v{i}.bin', (-1,1024)) for i in range(n)]